In [4]:
import sys
import pandas as pd
import numpy as np
from pandas.io import sql
from pandas.io.sql import read_sql
from pandas.io.sql import to_sql

#sqlalchemy
import sqlalchemy
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData, ForeignKey

# for postgres
import psycopg2

# suppress scientific notation
pd.options.display.float_format = '{:.2f}'.format

print('OK')

OK


In [2]:
# open a new connection to pyanalysis
engine = create_engine('postgresql://postgres@localhost:5432/pyanalysis')
print("OK")

OK


In [6]:
# let's see if we can do math with sql

sql = "SELECT 11/6"

df = pd.read_sql(sql, engine)

df

?column?
0         1

In [7]:
# however, it does not always work
# for instance, we can't do 11%6, which should return the remainder 5
# we may need a cursor, and a different engine
engine = psycopg2.connect(dbname='postgres',
      user='tbroderick', host='localhost')
cur = engine.cursor()
print('ready')

ready


In [13]:
# simple division of integers
cur.execute("SELECT 11/6")
ans= cur.fetchall()
print("division of integers: ",ans)

# the remainder
cur.execute("SELECT 11%6")
ans= cur.fetchall()
print("remainder: ",ans)

# now get it back as a numeric
cur.execute("SELECT 11.0/6")
ans= cur.fetchall()
print("as numeric: ",ans)

division of integers:  [(1,)]
remainder:  [(5,)]
as numeric:  [(Decimal('1.8333333333333333'),)]


In [25]:
# exponentiation
cur.execute("SELECT 3 ^ 4")
# 3 x 3 x 3 x 3 is 81
ans= cur.fetchall()
print("exponentiation: ",ans)

# square root (operator)
cur.execute("SELECT |/ 9")
# because 3 x 3 is 9
ans= cur.fetchall()
print("square root (operator): ",ans)

# square root (function)
cur.execute("SELECT sqrt(10)")
# returns pie
ans= cur.fetchall()
print("square root (function): ",ans)

# cube root
cur.execute("SELECT ||/ 27")
# the cube root of 27 is 3 (3x3x3)
ans= cur.fetchall()
print("cube root: ",ans)

# factorial
cur.execute("SELECT 4 !")
# 1 x 2 x 3 x 4 = 24
ans= cur.fetchall()
print("square root (function): ",ans)

exponentiation:  [(81.0,)]
square root (operator):  [(3.0,)]
square root (function):  [(3.16227766016838,)]
cube root:  [(3.0,)]
square root (function):  [(Decimal('24'),)]


In [33]:
# now in python

# simple division of integers
print( "integer division: ",int(11/6) )

# the remainder
print( "remainder: ",int(11%6) )

# now get it back as a numeric
print( "as numeric: ",round(11/6,2) )

# exponentiation
print( "exponentiation: ",3**4 )

# square root (function)
print( "square root (function): ",np.sqrt(10) )

# cube root
print( "cube root: ",27**(1/3) )
# the cube root of 27 is 3 (3x3x3)

# factorial
import math
print( "factorial: ",math.factorial(4) )


integer division:  1
remainder:  5
as numeric:  1.83
exponentiation:  81
square root (function):  3.1622776601683795
cube root:  3.0
factorial:  24


In [5]:
# how about finding the sum, average, median and mode of a column?

# open a new connection to pyanalysis
engine = create_engine('postgresql://postgres@localhost:5432/pyanalysis')
print("OK")

sql = """
SELECT p0010001 as "County" FROM us_counties_2010
"""
df = pd.read_sql(sql, engine)
# summary stats
df.describe()

OK


County
count    3143.00
mean    98232.75
std    312901.20
min        82.00
25%     11104.50
50%     25857.00
75%     66699.00
max   9818605.00

In [9]:
print("Sum: ",df['County'].sum() )
print("mean: ",round(df['County'].mean(),2) )
print("median: ",df['County'].median())
print("mode: ",df['County'].mode())

Sum:  308745538
mean:  98232.75
median:  25857.0
mode:  0    21720
dtype: int64


In [35]:
# performing math on joining rows
# open a new connection to pyanalysis
engine = create_engine('postgresql://postgres@localhost:5432/pyanalysis')
print("OK")

sql = """
    SELECT c2010.geo_name,
           c2010.state_us_abbreviation AS state,
           c2010.p0010001 AS pop_2010,
           c2000.p0010001 AS pop_2000,
           c2010.p0010001 - c2000.p0010001 AS raw_change,
           round( (CAST(c2010.p0010001 AS numeric(8,1)) - c2000.p0010001)
               / c2000.p0010001 * 100, 1 ) AS pct_change
    FROM us_counties_2010 c2010 INNER JOIN us_counties_2000 c2000
    ON c2010.state_fips = c2000.state_fips
       AND c2010.county_fips = c2000.county_fips
       AND c2010.p0010001 <> c2000.p0010001
    ORDER BY pct_change DESC
"""

df = pd.read_sql(sql, engine)

# what we should see is the result of an inner join, which returns only those matching criteria
# state fips, county fips and where population is not the same
# returns only certain tables
# from 2010 table: geo_name, state, pop
# from 2000 table: pop
# raw_change and pct_change are calculated
# with pct_change rounded to 1 decimal

df.head(20)

OK


geo_name state  pop_2010  pop_2000  raw_change  pct_change
0      Kendall County    IL    114736     54544       60192       110.4
1        Pinal County    AZ    375770    179727      196043       109.1
2      Flagler County    FL     95696     49832       45864        92.0
3      Lincoln County    SD     44828     24131       20697        85.8
4      Loudoun County    VA    312311    169599      142712        84.1
5     Rockwall County    TX     78337     43080       35257        81.8
6      Forsyth County    GA    175511     98407       77104        78.4
7       Sumter County    FL     93420     53345       40075        75.1
8     Paulding County    GA    142324     81678       60646        74.3
9     Sublette County    WY     10247      5920        4327        73.1
10       Henry County    GA    203922    119341       84581        70.9
11       Teton County    ID     10170      5999        4171        69.5
12  Williamson County    TX    422679    249967      172712        69.1
13   Fort Bend County    TX    585375    354452      230923        65.1
14       Union County    NC    201292    123677       77615        62.8
15     Douglas County    CO    285465    175766      109699        62.4
16      Dallas County    IA     66135     40750       25385        62.3
17      Newton County    GA     99958     62001       37957        61.2
18        Hays County    TX    157107     97589       59518        61.0
19      Collin County    TX    782341    491675      290666        59.1

In [36]:
# how we'd do that in python, pandas
sql = """
    SELECT c2010.geo_name,
           c2010.state_us_abbreviation AS state,
           c2010.p0010001 AS pop_2010,
           c2000.p0010001 AS pop_2000
    FROM us_counties_2010 c2010 INNER JOIN us_counties_2000 c2000
    ON c2010.state_fips = c2000.state_fips
       AND c2010.county_fips = c2000.county_fips
       AND c2010.p0010001 <> c2000.p0010001
"""

df = pd.read_sql(sql, engine)

df.head(5)

geo_name state  pop_2010  pop_2000
0  Autauga County    AL     54571     43671
1  Baldwin County    AL    182265    140415
2  Barbour County    AL     27457     29038
3     Bibb County    AL     22915     20826
4   Blount County    AL     57322     51024

In [51]:

df['raw_change'] = df['pop_2010'] - df['pop_2000']
df['pct_change'] = round( ( (df['pop_2010'] - df['pop_2000'])/df['pop_2000'])*100,2 )
df = df.sort_values(['pct_change'], ascending=False).reset_index(drop=True)
df

geo_name state  pop_2010  pop_2000  pct_change  \
0               Kendall County    IL    114736     54544      110.35   
1                 Pinal County    AZ    375770    179727      109.08   
2               Flagler County    FL     95696     49832       92.04   
3               Lincoln County    SD     44828     24131       85.77   
4               Loudoun County    VA    312311    169599       84.15   
5              Rockwall County    TX     78337     43080       81.84   
6               Forsyth County    GA    175511     98407       78.35   
7                Sumter County    FL     93420     53345       75.12   
8              Paulding County    GA    142324     81678       74.25   
9              Sublette County    WY     10247      5920       73.09   
10                Henry County    GA    203922    119341       70.87   
11                Teton County    ID     10170      5999       69.53   
12           Williamson County    TX    422679    249967       69.09   
13            Fort Bend County    TX    585375    354452       65.15   
14                Union County    NC    201292    123677       62.76   
15              Douglas County    CO    285465    175766       62.41   
16               Dallas County    IA     66135     40750       62.29   
17               Newton County    GA     99958     62001       61.22   
18                 Hays County    TX    157107     97589       60.99   
19               Collin County    TX    782341    491675       59.12   
20             Delaware County    OH    174214    109989       58.39   
21             Franklin County    WA     78163     49347       58.39   
22               Forest County    PA      7716      4946       56.00   
23              Osceola County    FL    268685    172493       55.77   
24           Montgomery County    TX    455746    293768       55.14   
25              Wasatch County    UT     23530     15215       54.65   
26            St. Johns County    FL    190039    123135       54.33   
27               Denton County    TX    662614    432976       53.04   
28           Washington County    UT    138115     90354       52.86   
29             Cherokee County    GA    214346    141903       51.05   
...                        ...   ...       ...       ...         ...   
3106             Blaine County    NE       478       583      -18.01   
3107             Emmons County    ND      3550      4331      -18.03   
3108  Yakutat City and Borough    AK       662       808      -18.07   
3109              Swift County    MN      9783     11956      -18.17   
3110            Greeley County    KS      1247      1534      -18.71   
3111            Quitman County    MS      8223     10117      -18.72   
3112               Lane County    KS      1750      2155      -18.79   
3113         Washington County    MS     51137     62977      -18.80   
3114          Claiborne County    MS      9604     11831      -18.82   
3115             Jewell County    KS      3077      3791      -18.83   
3116           McDowell County    WV     22113     27329      -19.09   
3117          Esmeralda County    NV       783       971      -19.36   
3118          Culberson County    TX      2398      2975      -19.39   
3119               King County    TX       286       356      -19.66   
3120             Monroe County    AR      8149     10254      -20.53   
3121             Tensas Parish    LA      5252      6618      -20.64   
3122          Jefferson County    MS      7726      9740      -20.68   
3123       Bristol Bay Borough    AK       997      1258      -20.75   
3124             Cottle County    TX      1505      1904      -20.96   
3125           Cimarron County    OK      2475      3148      -21.38   
3126             Towner County    ND      2246      2876      -21.91   
3127              Kiowa County    KS      2553      3278      -22.12   
3128           Sheridan County    ND      1321      1710      -22.75   
3129      Chattahoochee County    GA     11267     14882      